In [ ]:
# use tensorflow 2.0
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import os
from tqdm import tqdm

In [ ]:
mobile_net_path = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
mobile_net = hub.KerasLayer(mobile_net_path,output_shape = [1280] , trainable=False)

In [ ]:
# old-fashioned way of managing data
def load_pictures(path):
    pictures = []
    os.chdir(path)
    print("changed working dir")
    picture_paths = np.array(os.listdir())
    picture_paths = shuffle(picture_paths)
    for path in tqdm(picture_paths[:1800]):
        pictures.append(mpimg.imread(path).astype(np.float16))
    pictures = np.array(pictures)
    pictures = np.expand_dims(pictures, axis=-1)
    return pictures
def shuffle(arr):
    l = len(arr)
    indices = np.arange(l)
    np.random.shuffle(indices)
    arr = arr[indices]
    return arr
def get_batch(data, batch_size, shuff = True):
    if shuff:
        data = shuffle(data)
    length = len(data)
    batches = length//batch_size
    for i in range(batches):
        batch = data[i*batch_size:(i+1)*batch_size]
        yield batch

# advanced data generator
# for mode,see https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory
def get_data_iterator(dir_path, mode, batch_size):
    datagen = K.preprocessing.image.ImageDataGenerator(dtype=tf.float16)
    data = datagen.flow_from_directory(dir_path, target_size = (1024,1024),color_mode = "grayscale",batch_size = batch_size,class_mode=mode)
    return data

In [ ]:
frame_rate = 15
batch_size = 10

In [ ]:
def interpolation_layer(X):
    X1 = tf.image.resize(X,(224,224),method=tf.image.ResizeMethod.BILINEAR)
    X2 = tf.image.resize(X,(224,224),method=tf.image.ResizeMethod.BICUBIC,antialias=True)
    X3 = tf.image.resize(X,(224,224),method=tf.image.ResizeMethod.GAUSSIAN)
    X = K.layers.Concatenate(axis=-1)([X1,X2,X3])
    return X

def encoder_layer(X):
    X = tf.image.resize(X,(512,512),method=tf.image.ResizeMethod.BILINEAR)
    X = K.layers.Conv2D(filters=1,kernel_size=1,strides=1,use_bias=True,padding="valid",name ="mask",kernel_regularizer=K.regularizers.l2(l=0.01))(X)
    X = K.layers.BatchNormalization(name="bias")(X)
    X = K.layers.Activation("relu")(X)
    # input shape = (512,512,1)
    X = K.layers.Conv2D(filters=4, kernel_size=(3,3), strides=(1,1),padding="valid",kernel_initializer="he_normal",kernel_regularizer=K.regularizers.l2(l=0.01))(X)
    X = K.layers.BatchNormalization(name="bn1")(X)
    X = K.layers.Activation("selu")(X)

    # input shape = (510,510,4)
    X = K.layers.MaxPool2D((2,2),name="max_pool1",padding="valid",strides=(2,2))(X)

    # input shape = (255,255,4)
    X = K.layers.Conv2D(filters=8,kernel_size=(4,4),strides=1,padding="valid",kernel_initializer="he_normal",kernel_regularizer=K.regularizers.l2(l=0.01))(X)
    X = K.layers.BatchNormalization(name="bn2")(X)
    X = K.layers.Activation("selu")(X)
    # input shape = (252,252,8)
    X = K.layers.MaxPool2D((2,2),name="max_pool2",padding="valid",strides=(1,1))(X)

    # input shape = (251,251,8)
    X = K.layers.Conv2D(filters=12,kernel_size=(5,5),strides=1,padding="valid",kernel_initializer="he_normal",kernel_regularizer=K.regularizers.l2(l=0.01))(X)
    X = K.layers.BatchNormalization(name="bn3")(X)
    X = K.layers.Activation("selu")(X)
    # input shape = (247,247,16)
    X = K.layers.MaxPool2D((2,2),name="max_pool3",padding="valid",strides=(1,1))(X)

    # input shape = (246,246,16)
    X = K.layers.Conv2D(filters=16,kernel_size=(7,7),strides=1,padding="valid",kernel_initializer="he_normal",kernel_regularizer=K.regularizers.l2(l=0.01))(X)
    X = K.layers.BatchNormalization(name="bn4")(X)
    X = K.layers.Activation("selu")(X)
    # input shape = (240,240,32)
    X = K.layers.MaxPool2D((2,2),name="max_pool4",padding="valid",strides=(1,1))(X)

    # input shape = (239,239,32)
    X = K.layers.Conv2D(filters=20,kernel_size=(8,8),strides=1,padding="valid",kernel_initializer="he_normal",kernel_regularizer=K.regularizers.l2(l=0.01))(X)
    X = K.layers.BatchNormalization(name="bn5")(X)
    X = K.layers.Activation("selu")(X)

    # input shape = (232,232,64)
    X = K.layers.Conv2D(filters=3,kernel_size=(9,9),strides=1,padding="valid",kernel_initializer="he_normal",kernel_regularizer=K.regularizers.l2(l=0.01))(X)
    X = K.layers.BatchNormalization(name="bn6")(X)
    X = K.layers.Activation("selu")(X)
    # output shape = (224,224,3)
    return X

def decoder_layer(X):
    # input shape = (224,224,3)
    D = tf.image.resize(X,size=(256,256))
    # input shape = (256,256,3)
    D = K.layers.Conv2DTranspose(filters=6,kernel_size=3,strides=2,padding="same",activation="relu")(D)
    # input shape = (512,512,6)
    D = K.layers.Conv2DTranspose(filters=1,kernel_size=3,strides=2,padding="same",activation="relu")(D)
    # output (1024,1024,1)
    return D

def rnn_layer(X):
    R = K.layers.LSTM(units=128,return_sequences=True)(X)
    R = K.layers.Dropout(rate=0.5)(R)
    R = K.layers.BatchNormalization(name="bn1_rnn")(R)
    
    R = K.layers.LSTM(units=128,return_sequences=True)(R)
    R = K.layers.Dropout(rate=0.5)(R)
    R = K.layers.BatchNormalization(name = "bn2_rnn")(R)
    R = K.layers.TimeDistributed(K.layers.Dense(3,activation="softmax"))(R)
    return R

In [ ]:
#(batch, frame, C, H, W)->(batch*frame, C, H, W)->CNN->(batch*frame, feature)->(batch, frame, feature)->RNN

In [ ]:
# recognizer_without_encoder
input_images = K.layers.Input(shape=(1024,1024,1),batch_size=batch_size*frame_rate)
X = interpolation_layer(input_images)
X = mobile_net(X)
X = tf.reshape(X,(-1,frame_rate,1280))
R = rnn_layer(X)
preds = tf.reshape(R,(-1,3))
recognizer_without_encoder = K.Model(inputs = input_images, outputs = preds, name="recognizer_without_encoder")

In [ ]:
recognizer_without_encoder.summary()

In [ ]:
# recogizer_with_encoder
input_images = K.layers.Input(shape=(1024,1024,1),batch_size=batch_size*frame_rate)
E = encoder_layer(input_images)
D = decoder_layer(E)
M = mobile_net(E)
X = tf.reshape(M,(-1,frame_rate,1280))
R = rnn_layer(X)
preds = tf.reshape(R,(-1,3))
recognizer_with_encoder = K.Model(inputs = input_images, outputs = preds, name ="recognizer_with_encoder" )
autoencoder = K.Model(inputs = input_images, outputs = D, name="autoencoder")

In [ ]:
autoencoder.summary()

In [ ]:
# for save and load, see https://www.tensorflow.org/tutorials/keras/save_and_load